# Data Cleaning
## 1. Check that data file exists and read it from CSV into dataframe

In [3]:
# Import and configuration
import numpy as np
import os
import pandas as pd
from   time import monotonic

from config import *

In [4]:
gun_violence_data = None

if not(os.path.exists(incidents_raw_data_path)):
    print(f'The incidents data file was not found at "{os.path.abspath(incidents_raw_data_path)}"')
    print(f'Make sure you\'ve downloaded it (instructions can be found at {os.path.abspath("../LoadData.md")})')
    exit()
else:
    time_taken = monotonic()
    print(f'Loading data from "{os.path.abspath(incidents_raw_data_path)}"...')
    gun_violence_data = pd.read_csv(incidents_raw_data_path)
    time_taken = monotonic() - time_taken
    print(f'Data was successfully loaded. It tooks {time_taken:.3f} seconds and there are {len(gun_violence_data.index)} rows')

Loading data from "C:\dev\gun-violence\Data\Autogenerated\gun-violence-data_01-2013_03-2018.csv"...
Data was successfully loaded. It tooks 2.578 seconds and there are 239677 rows


## 2. Drop Columns not Required

In [5]:
# First lets take a look a the columns we have
for column_name in gun_violence_data.columns:
    print(column_name)

incident_id
date
state
city_or_county
address
n_killed
n_injured
incident_url
source_url
incident_url_fields_missing
congressional_district
gun_stolen
gun_type
incident_characteristics
latitude
location_description
longitude
n_guns_involved
notes
participant_age
participant_age_group
participant_gender
participant_name
participant_relationship
participant_status
participant_type
sources
state_house_district
state_senate_district


In [6]:
# Now lets take only those that we need
clean_gun_violence_data = gun_violence_data.loc[:,[
    'date',
    'state',
    'n_killed',
    'n_injured',
    'incident_characteristics',
    'latitude',
    'longitude',
    'participant_age',
    'participant_age_group',
    'participant_gender',
    'participant_relationship',
    'participant_status',
    'participant_type'    
]]

## 3. Rename Columns 

In [7]:
clean_gun_violence_data = clean_gun_violence_data.rename(columns={
    'date': 'Year',
    'state': 'State Name',
    'n_killed': 'Number of Killed',
    'n_injured': 'Number of Injured',
    'incident_characteristics': 'Incident Info',
    'latitude': 'Latitude',
    'longitude': 'Longitude',
    'participant_age': 'Participant Age',
    'participant_age_group': 'Participant Age Group',
    'participant_gender': 'Participant Gender',
    'participant_relationship': 'Participant Relationship',
    'participant_status': 'Participant Status',
    'participant_type': 'Participant Type'
})

## 4. Change Column Types

In [8]:
clean_gun_violence_data.loc[:,'Year'] = clean_gun_violence_data.loc[:,'Year'].apply(lambda x: int(x[:4]))

## 5. Enrich Dataset with Extra Columns

In [9]:
states_data = pd.read_csv(states_data_path)
states_data = states_data.set_index('name', drop=True)
states_data.head()

,abbreviation
name,
Alabama,AL
Alaska,AK
Arizona,AZ
Arkansas,AR
California,CA


In [10]:
clean_gun_violence_data['State Abbr'] = clean_gun_violence_data['State Name'].copy()
clean_gun_violence_data.loc[:,'State Abbr'] = clean_gun_violence_data.loc[:,'State Abbr'].map(states_data['abbreviation'])
clean_gun_violence_data.head()

,Year,State Name,Number of Killed,Number of Injured,Incident Info,Latitude,Longitude,Participant Age,Participant Age Group,Participant Gender,Participant Relationship,Participant Status,Participant Type,State Abbr
0,2013,Pennsylvania,0,4,Shot - Wounded/Injured||Mass Shooting (4+ vict...,40.3467,-79.8559,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||3::Male||4::Female,NaN,0::Arrested||1::Injured||2::Injured||3::Injure...,0::Victim||1::Victim||2::Victim||3::Victim||4:...,PA
1,2013,California,1,3,"Shot - Wounded/Injured||Shot - Dead (murder, a...",33.9090,-118.3330,0::20,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male,NaN,0::Killed||1::Injured||2::Injured||3::Injured,0::Victim||1::Victim||2::Victim||3::Victim||4:...,CA
2,2013,Ohio,1,3,"Shot - Wounded/Injured||Shot - Dead (murder, a...",41.4455,-82.1377,0::25||1::31||2::33||3::34||4::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Male||1::Male||2::Male||3::Male||4::Male,NaN,"0::Injured, Unharmed, Arrested||1::Unharmed, A...",0::Subject-Suspect||1::Subject-Suspect||2::Vic...,OH
3,2013,Colorado,4,0,"Shot - Dead (murder, accidental, suicide)||Off...",39.6518,-104.8020,0::29||1::33||2::56||3::33,0::Adult 18+||1::Adult 18+||2::Adult 18+||3::A...,0::Female||1::Male||2::Male||3::Male,NaN,0::Killed||1::Killed||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,CO
4,2013,North Carolina,2,2,"Shot - Wounded/Injured||Shot - Dead (murder, a...",36.1140,-79.9569,0::18||1::46||2::14||3::47,0::Adult 18+||1::Adult 18+||2::Teen 12-17||3::...,0::Female||1::Male||2::Male||3::Female,3::Family,0::Injured||1::Injured||2::Killed||3::Killed,0::Victim||1::Victim||2::Victim||3::Subject-Su...,NC


## 6. Export Clean Data to CSV

In [11]:
#clean_gun_violence_data.to_csv(incidents_clean_data_path, index=False)
#print(f'Clean data was successfully expored to "{os.path.abspath(incidents_clean_data_path)}"')

Clean data was successfully expored to "C:\dev\gun-violence\Data\Autogenerated\gun-violence-data_01-2013_03-2018_clean.csv"
